# CrewAI Demo

This notebook demonstrates **CrewAI**, a comprehensive framework for orchestrating role-based multi-agent teams that mimic real-world human crew dynamics.

## Required Environment Variables

```
OPENAI_API_KEY    # Your OpenAI API key (required)
SERPER_API_KEY    # Your Serper API key (optional, for web search)
```

Please refer to the [README](README.md) for instructions on setting up environment variables.


In [ ]:
# If running standalone without the project setup:
# pip install crewai crewai-tools openai python-dotenv

In [7]:
import os
import yaml
from pathlib import Path
from dotenv import load_dotenv
from typing import Any, Type, Optional
from pydantic import BaseModel, Field

# Load environment variables
load_dotenv(override=True)

# Import CrewAI components
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from crewai.tools import BaseTool

### CrewAI Core Components Loaded:
- **Agent:** Role-based AI team members
- **Task:** Structured work items
- **Crew:** Team orchestration
- **Process:** Execution workflows

## 🎯 What We're Building

In this demo, we'll create a **Content Marketing Team** using CrewAI to demonstrate how AI agents can work together like a real marketing department.

### The Scenario
We're building an automated workflow to create a blog post about "AI-powered content marketing" - a task that typically requires:
- **Research** → gathering market insights and trends
- **Writing** → transforming research into engaging content  
- **Editing** → polishing for quality and SEO


## 🛠️ Tool Setup

CrewAI supports various tools to extend agent capabilities. In this demo, we'll use the **SerperDev** tool for web search, which allows agents to gather real-time information from the internet.

### Web Search Tool:
- **SerperDevTool** - Searches the web for current information
- Requires `SERPER_API_KEY` environment variable
- If no API key is provided, agents will use their LLM knowledge instead

This demonstrates how CrewAI agents can integrate with external APIs to enhance their capabilities beyond the base LLM knowledge.

In [3]:
# Initialize the web search tool for research
print("Initializing CrewAI Tools...")
print("=" * 50)

# Research Tool - SerperDev for web search (requires API key)
web_search_tool = None
try:
    if os.getenv('SERPER_API_KEY'):
        web_search_tool = SerperDevTool()
        print("✓ Web Search Tool (SerperDev) - Initialized with API key")
    else:
        print("✗ Web Search Tool - No SERPER_API_KEY found")
        print("  Agents will use LLM knowledge without web search capability")
except Exception as e:
    print(f"✗ Web Search Tool - Error: {e}")
    print("  Agents will proceed without web search")

print("=" * 50)

if web_search_tool:
    print("Tool available: Web Search (powered by Serper)")
else:
    print("No external tools available - agents will rely on LLM capabilities")

Initializing CrewAI Tools...
✓ Web Search Tool (SerperDev) - Initialized with API key
Tool available: Web Search (powered by Serper)


## 🤖 Agent Definition

CrewAI agents are **role-based team members** with comprehensive personas. Each agent has:

### Core Agent Attributes:
- **Role** - The agent's job title/specialization
- **Goal** - What the agent aims to achieve
- **Backstory** - Context and expertise background
- **Tools** - Available capabilities and integrations
- **LLM** - The underlying language model


In [8]:
# Create agents.yaml configuration - Shows agent structure
agents_yaml_content = """
content_researcher:
  role: Senior Content Researcher
  goal: Research and gather comprehensive information on given topics to support content creation
  backstory: You are an experienced researcher with 10+ years in digital marketing research. You excel at finding credible sources, identifying market trends, and extracting actionable insights from complex data.
  tools:
  - web_search         # SerperDevTool
  verbose: true
  allow_delegation: false

content_writer:
  role: Senior Content Writer
  goal: Create engaging, high-quality content based on research and brand guidelines
  backstory: You are a seasoned content writer with expertise in creating compelling marketing content. You understand how to transform research into engaging narratives that resonate with target audiences.
  tools: []            # No tools - uses LLM capabilities
  verbose: true
  allow_delegation: false

content_editor:
  role: Senior Content Editor
  goal: Review, refine, and ensure content quality meets brand standards and objectives
  backstory: You are an experienced content editor with a keen eye for detail, brand consistency, and audience engagement. You excel at polishing content to perfection while maintaining the original message. As a senior editor, you can delegate specific editing tasks when needed.
  tools: []            # No tools - uses LLM capabilities and delegation
  verbose: true
  allow_delegation: true  # Can delegate to other agents for specialized tasks
"""

# Parse YAML for documentation purposes
agents_config = yaml.safe_load(agents_yaml_content.strip())

print("Agent Configuration (YAML):")
print("=" * 50)
for agent_name, config in agents_config.items():
    print(f"\n{agent_name}:")
    print(f"  Role: {config['role']}")
    tools = config.get('tools', [])
    if tools:
        print(f"  Tools: {', '.join(tools)}")
    else:
        print(f"  Tools: None (LLM only)")
    print(f"  Delegation: {'✅ Enabled (can delegate tasks)' if config.get('allow_delegation', False) else '❌ Disabled (individual contributor)'}")
print("=" * 50)
print("\n📌 Delegation Feature:")
print("When allow_delegation is True, agents can delegate subtasks to other agents.")
print("This enables hierarchical workflows where senior agents coordinate work.")

Agent Configuration (YAML):

content_researcher:
  Role: Senior Content Researcher
  Tools: web_search
  Delegation: ❌ Disabled (individual contributor)

content_writer:
  Role: Senior Content Writer
  Tools: None (LLM only)
  Delegation: ❌ Disabled (individual contributor)

content_editor:
  Role: Senior Content Editor
  Tools: None (LLM only)
  Delegation: ✅ Enabled (can delegate tasks)

📌 Delegation Feature:
When allow_delegation is True, agents can delegate subtasks to other agents.
This enables hierarchical workflows where senior agents coordinate work.


In [9]:
# Create Agent instances from YAML configuration

# Tool mapping - map tool names from YAML to actual tool objects
tools_map = {
    'web_search': web_search_tool
}

# Create agents dynamically from YAML configuration
agents = {}
for agent_name, config in agents_config.items():
    # Map tool names to actual tool objects
    agent_tools = []
    for tool_name in config.get('tools', []):
        if tool_name in tools_map and tools_map[tool_name]:
            agent_tools.append(tools_map[tool_name])

    # Create agent with configuration from YAML
    agents[agent_name] = Agent(
        role=config['role'],
        goal=config['goal'],
        backstory=config['backstory'],
        tools=agent_tools,
        verbose=config.get('verbose', True),
        allow_delegation=config.get('allow_delegation', False)
    )

# Extract agents for easy reference
researcher = agents['content_researcher']
writer = agents['content_writer']
editor = agents['content_editor']

# Display agent configuration
print("Content Marketing Team Assembled!")
print("=" * 50)
for agent_name, agent in agents.items():
    print(f"\n{agent.role}:")
    print(f"  Goal: {agent.goal[:60]}...")
    print(f"  Tools: {len(agent.tools)} available")
    if agent.tools:
        for tool in agent.tools:
            tool_name = getattr(tool, 'name', 'Web Search')
            print(f"    - {tool_name}")
    else:
        print(f"    - No tools (relies on LLM capabilities)")
    print(f"  Delegation: {'✅ Can delegate tasks' if agent.allow_delegation else '❌ Individual contributor'}")
print("=" * 50)

print("\n📌 Configuration-Driven Design:")
print("Agents are created directly from the YAML configuration above,")
print("making it easy to modify team composition without changing code.")

Content Marketing Team Assembled!

Senior Content Researcher:
  Goal: Research and gather comprehensive information on given topic...
  Tools: 1 available
    - Search the internet with Serper
  Delegation: ❌ Individual contributor

Senior Content Writer:
  Goal: Create engaging, high-quality content based on research and ...
  Tools: 0 available
    - No tools (relies on LLM capabilities)
  Delegation: ❌ Individual contributor

Senior Content Editor:
  Goal: Review, refine, and ensure content quality meets brand stand...
  Tools: 0 available
    - No tools (relies on LLM capabilities)
  Delegation: ✅ Can delegate tasks

📌 Configuration-Driven Design:
Agents are created directly from the YAML configuration above,
making it easy to modify team composition without changing code.


## ✅ Agents Successfully Created!

### Team Composition:

1. **Senior Content Researcher**
   - Goal: Research and gather comprehensive information
   - Tools: Web search (if API key available)
   - Delegation: ❌ Individual contributor

2. **Senior Content Writer**
   - Goal: Create engaging, high-quality content
   - Tools: None (uses LLM capabilities)
   - Delegation: ❌ Individual contributor

3. **Senior Content Editor**
   - Goal: Review and ensure content quality
   - Tools: None (uses LLM capabilities)
   - Delegation: ✅ **Can delegate tasks** (hierarchical workflows enabled)

### 📌 Delegation in CrewAI

**What is Delegation?**
When `allow_delegation=True`, an agent can:
- Break down complex tasks into subtasks
- Assign subtasks to other agents best suited for them
- Coordinate work across multiple agents
- Enable hierarchical team structures

**Example Use Cases:**
- A Senior Editor delegating fact-checking to the Researcher
- A Project Manager distributing tasks across specialists
- A Lead Developer assigning code reviews to team members

**Note**: The web search tool requires a SERPER_API_KEY. Without it, agents rely solely on their LLM knowledge.

## 📋 Task Definition & Dependencies

CrewAI tasks are **structured work items** that define what needs to be accomplished. Tasks have:

### Core Task Attributes:
- **Description** - What needs to be done
- **Agent** - Who will execute the task
- **Expected Output** - Success criteria
- **Context** - Dependencies on other tasks
- **Tools** - Task-specific capabilities

### Task Workflow Patterns:
- **Sequential**: Tasks executed in order
- **Parallel**: Independent tasks run simultaneously
- **Conditional**: Tasks based on previous outcomes

In [ ]:
# Create tasks.yaml configuration - Shows task dependencies
# Note: CrewAI Task class doesn't support estimated_duration - use description for timing expectations
tasks_yaml_content = """
research_task:
  description: Research the latest trends and best practices in AI-powered content marketing for 2024. Focus on practical applications, case studies, and emerging technologies.
  agent: content_researcher
  expected_output: A comprehensive research report with key trends, statistics, case studies, and actionable insights for AI content marketing.
  tools_used:
  - web_search         # For finding current information

writing_task:
  description: Create an engaging blog post about AI-powered content marketing based on the research findings. The post should be informative, engaging, and targeted at marketing professionals.
  agent: content_writer
  expected_output: A well-structured 1200-1500 word blog post with introduction, key sections, examples, and conclusion. Include engaging headlines and clear takeaways.
  context:
  - research_task
  tools_used: []       # Uses LLM capabilities

editing_task:
  description: Review and refine the blog post for clarity, engagement, brand consistency, and SEO optimization. Ensure the content meets quality standards and brand guidelines.
  agent: content_editor
  expected_output: A polished, publication-ready blog post with improved flow, corrected grammar, optimized headlines, and SEO enhancements.
  context:
  - research_task
  - writing_task
  tools_used: []       # Uses LLM capabilities and delegation
"""

# Parse YAML for documentation
tasks_config = yaml.safe_load(tasks_yaml_content.strip())

print("Task Configuration:")
print("=" * 50)
for task_name, config in tasks_config.items():
    print(f"\n{task_name}:")
    print(f"  Agent: {config['agent']}")
    tools_used = config.get('tools_used', [])
    if tools_used:
        print(f"  Tools Used: {', '.join(tools_used)}")
    else:
        print(f"  Tools Used: None")
    print(f"  Dependencies: {', '.join(config.get('context', ['None']))}")
print("=" * 50)

Task Configuration:

research_task:
  Agent: content_researcher
  Tools Used: web_search
  Dependencies: None

writing_task:
  Agent: content_writer
  Tools Used: LLM only
  Dependencies: research_task

editing_task:
  Agent: content_editor
  Tools Used: LLM only
  Dependencies: research_task, writing_task


### tasks.yaml Configuration

**YAML Configuration loaded successfully!**  
- **Tasks defined:** 3

**Task Workflow & Dependencies:**

📌 **Research Task**
- Agent: content_researcher
- Dependencies: None

📌 **Writing Task**
- Agent: content_writer
- Dependencies: research_task

📌 **Editing Task**
- Agent: content_editor
- Dependencies: research_task, writing_task

In [ ]:
# Create Task instances with dependencies

# Task 1: Research (no dependencies)
research_task = Task(
    description="""Research the latest trends and best practices in AI-powered content marketing for 2024.
    Focus on practical applications, case studies, and emerging technologies that marketing teams can implement.

    Key areas to investigate:
    - AI content generation tools and platforms
    - Personalization and audience targeting
    - Performance metrics and ROI measurement
    - Case studies of successful implementations
    - Future trends and predictions""",
    agent=researcher,
    expected_output="""A comprehensive research report containing:
    - Executive summary of key trends
    - 5-7 major trend categories with detailed analysis
    - At least 3 real-world case studies
    - Statistical data and market insights
    - Actionable recommendations for marketing teams"""
)

# Task 2: Writing (depends on research)
writing_task = Task(
    description="""Create an engaging and informative blog post about AI-powered content marketing
    based on the research findings. The content should be targeted at marketing professionals
    and business leaders looking to implement AI in their content strategies.

    Content requirements:
    - Compelling headline and introduction
    - Clear structure with subheadings
    - Include specific examples and case studies
    - Actionable tips and recommendations
    - Professional yet engaging tone
    - 1200-1500 words""",
    agent=writer,
    expected_output="""A well-structured blog post containing:
    - Attention-grabbing headline
    - Engaging introduction (100-150 words)
    - 4-6 main sections with clear subheadings
    - Real-world examples and case studies
    - Actionable takeaways for readers
    - Strong conclusion with call-to-action
    - 1200-1500 words total""",
    context=[research_task]  # This task depends on research completion
)

# Task 3: Editing (depends on both research and writing)
editing_task = Task(
    description="""Review and refine the blog post to ensure it meets high-quality standards
    for publication. Focus on clarity, engagement, brand consistency, and SEO optimization.

    Editing checklist:
    - Grammar, spelling, and punctuation accuracy
    - Flow and readability improvements
    - Brand voice and tone consistency
    - SEO optimization (keywords, meta descriptions)
    - Fact-checking and source verification
    - Call-to-action effectiveness""",
    agent=editor,
    expected_output="""A polished, publication-ready blog post with:
    - Perfect grammar and spelling
    - Improved flow and readability
    - Consistent brand voice
    - SEO-optimized headlines and structure
    - Verified facts and properly cited sources
    - Strong call-to-action
    - Editorial notes on changes made""",
    context=[research_task, writing_task]  # Depends on both previous tasks
)

print(f"\nTask Workflow:")
print(f"  1. {research_task.description.split('.')[0]}...")
print(f"     Agent: {research_task.agent.role}")
print(f"     Dependencies: None (starting task)")

print(f"\n  2. {writing_task.description.split('.')[0]}...")
print(f"     Agent: {writing_task.agent.role}")
print(f"     Dependencies: Research Task")

print(f"\n  3. {editing_task.description.split('.')[0]}...")
print(f"     Agent: {editing_task.agent.role}")
print(f"     Dependencies: Research Task + Writing Task")

## 👥 Crew Orchestration & Processes

The **Crew** is CrewAI's core orchestration component that manages how agents collaborate to complete tasks. 

### Crew Process Types:

1. **Sequential** - Tasks executed one after another (most common)
2. **Hierarchical** - Manager agent coordinates subordinates
3. **Consensual** - Agents collaborate to reach consensus

### Human Crew Abstraction:

CrewAI mimics real-world team dynamics:
- **Role Clarity**: Everyone knows their responsibilities
- **Collaboration**: Agents share context and build on each other's work
- **Delegation**: Senior roles can delegate to juniors
- **Quality Control**: Review and refinement cycles
- **Memory**: Teams remember past interactions and decisions

In [ ]:
# Create the Content Marketing Crew
content_crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research_task, writing_task, editing_task],
    process=Process.sequential,  # Tasks executed in dependency order
    verbose=True,  # Enable detailed logging
    memory=False  # Disabled to keep demo simple (context still passed via task dependencies)
)

print("Content Marketing Crew Assembled!")
print(f"\nCrew Configuration:")
print(f"  • Team Size: {len(content_crew.agents)} agents")
print(f"  • Workload: {len(content_crew.tasks)} tasks")
print(f"  • Process: {content_crew.process.value} execution")
print(f"  • Memory: {'Enabled' if content_crew.memory else 'Disabled (using explicit context passing)'}")
print(f"  • Verbose: {'Enabled' if content_crew.verbose else 'Disabled'}")

print(f"\nTeam Hierarchy & Roles:")
for i, agent in enumerate(content_crew.agents, 1):
    delegation_status = "Can delegate" if agent.allow_delegation else "Individual contributor"
    print(f"  {i}. {agent.role}")
    print(f"     Status: {delegation_status}")
    print(f"     Focus: {agent.goal[:50]}...")

print(f"\nWorkflow Process:")
for i, task in enumerate(content_crew.tasks, 1):
    dependencies = len(task.context) if hasattr(task.context, '__len__') else 0
    print(f"  Step {i}: {task.agent.role}")
    print(f"          Dependencies: {dependencies} task(s)")
    print(f"          Output: {task.expected_output.split('.')[0]}...")

## 🚀 Complete Workflow Demo: Marketing Content Creation

Now let's see our **Content Marketing Crew** in action! This is where CrewAI's power really shines - watching specialized agents collaborate like a real marketing team.

### Workflow Overview:
1. **Research Phase**: Senior Researcher gathers market intelligence
2. **Creation Phase**: Content Writer transforms research into engaging content
3. **Refinement Phase**: Editor polishes and optimizes for publication

### Expected Collaboration Patterns:
- **Context Sharing**: Each agent builds on previous work
- **Quality Escalation**: Each step improves the output
- **Role Specialization**: Agents focus on their expertise

Let's execute our crew and observe the human-like collaboration!

In [ ]:
print("Starting Content Marketing Crew Execution...")
print("=" * 70)

result = content_crew.kickoff()

print("\nCrew Workflow Completed Successfully!")
print("=" * 70)
print("FINAL DELIVERABLE")
print("=" * 70)
print(result)
print("=" * 70)

## 📋 Summary

This notebook demonstrated **CrewAI's comprehensive approach** to multi-agent AI systems through hands-on examples.

### 🤖 **Agents** (Role-Based Team Members)
✅ **Comprehensive personas** - Role, goal, backstory, tools  
✅ **YAML configuration** - Scalable team management  
✅ **Specialization** - Domain experts with focused capabilities  
✅ **Delegation support** - Hierarchical team structures  

### 📋 **Tasks** (Structured Work Items)
✅ **Clear specifications** - Description, expected output, context  
✅ **Dependency management** - Sequential and parallel workflows  
✅ **Context sharing** - Tasks build on previous work  
✅ **Quality gates** - Review and refinement cycles  

### 👥 **Crew** (Team Orchestration)
✅ **Process types** - Sequential, hierarchical, consensual  
✅ **Memory management** - Persistent crew knowledge  
✅ **Human integration** - Human-in-the-loop workflows  
✅ **Production features** - Monitoring, error handling, scalability  

## 📚 **Resources**

- **GitHub**: [crewAI-Inc/crewAI](https://github.com/crewAI-Inc/crewAI)
- **Documentation**: [CrewAI Docs](https://docs.crewai.com/)
- **CrewAI Studio**: Visual workflow builder
- **Community**: Discord and forum support

**CrewAI represents the future of enterprise multi-agent AI - role-based, scalable, and built for real-world team collaboration! 🚀**